# ROOT analyser

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import cv2 as cv
import numpy as np

from src.viz.graphs import draw_map
from src.viz.images import imshow, scaled_imshow
from src.utils import crop_contour, rotate_image, warp_contour, calculate_color_percentage, resize_contour
from src.file_reading import read_pdf
from src.detection.elements import descriptor_detect, detect_dice_tray, detect_score_board, detect_buildings
from src.detection.game import calculate_current_score
from src.get_first_frame import get_first_frame
from src.tracking.algorithms import track_video
from src.tracking.Card import Card,CardPile
from src.tracking.ScoreBoard import ScoreBoard
from src.tracking.Buildings import Buildings
from src.tracking.Board import Board
from src.tracking.Dice import Dice

In [ ]:
large_imshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.8, fy=0.8)
simshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.3, fy=0.3)
mini_imshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.2, fy=0.2)

In [ ]:
mse = lambda a_1,a_2: np.mean((np.array(a_1)-np.array(a_2))**2)
_max_color_diff = mse((0,0,0),(255,255,255))
def color_sim(a_1,a_2):
    return 1 - (mse(a_1,a_2)/_max_color_diff)

In [ ]:
clip = lambda idx: f"clip_{idx}"
clip_mp4 = lambda idx: f"{clip(idx)}.mp4"

## Data

The input data was divided into 3 groups depending on the difficulty for example:
- easy: perfect top-down view, the game elements are not covered with hands when carrying them, the lighting is good
- medium: strong light at the side causing shadows,
- difficult: same as medium + a slightly angled camera, hands covering the pieces

There are 3 clips per difficulty. The data is located in a [Google Drive]((https://drive.google.com/drive/folders/1VrQ98TC5jPmWk1QYr3lUP3SGk_3_AEmx?usp=sharing)). We also resized the clips using the resize_data.py script, to speed up the detection process.


In [ ]:
DATA_DIR = "./data"
DIFFICULTIES = ["easy", "medium", "hard"]
CLIP_DIRS = dict([(diff, f"{DATA_DIR}/{diff}") for diff in DIFFICULTIES])
RESIZED_CLIP_DIRS = dict([(diff, f"{DATA_DIR}/{diff}/resized") for diff in DIFFICULTIES])
FIRST_FRAMES = dict([(diff,get_first_frame(f"{_dir}/{clip_mp4(0)}")) for diff, _dir in CLIP_DIRS.items()])
RESIZED_FIRST_FRAMES = dict([(diff,get_first_frame(f"{_dir}/{clip_mp4(0)}")) for diff, _dir in RESIZED_CLIP_DIRS.items()])
LOWER_ORANGE = np.array([0, 100, 100])
ORANGE = (48,91,198)
UPPER_ORANGE = np.array([20, 255, 255])
LOWER_DARK_BLUE = np.array([100, 50, 50])
BLUE = (92,38,15)
UPPER_DARK_BLUE = np.array([140, 255, 255])

In [ ]:
mini_imshow(FIRST_FRAMES["easy"])

In [ ]:
mini_imshow(FIRST_FRAMES["medium"])

In [ ]:
mini_imshow(FIRST_FRAMES["hard"])

In [ ]:
simshow(np.concatenate([RESIZED_FIRST_FRAMES[diff] for diff in DIFFICULTIES], axis=1))

The game is played between 2 factions: Eyrie Dynasties (blue birds), Marquise de Cat (orange cats). The board is a Winter Map. Because the clearings in the forest are barely differentiable, a mask was created to help with detecting static elements of the board. 

In [ ]:
GAME_DATA_DIR = f"{DATA_DIR}/game_data"

In [ ]:
board_mask = cv.imread(f"{GAME_DATA_DIR}/board_mask.png")

simshow(board_mask)

- The red indicates the where the score track is. 
- The green defines where craftable items are.
- The blue shows where the clearing approximately are, with the black squares showing where building spaces are.

JSON was created to define paths on the map, done purely for drawing a graph of the map.

In [ ]:
with open(f"{GAME_DATA_DIR}/board_info.json", "r") as info_file:
    board_info = json.load(info_file)

draw_map(board_info)

To help with detection a print and play set is used with all the elements taken from [PnP PARADISE](https://www.pnpparadise.com/set1/root).

In [ ]:
board_ref = read_pdf(f"{GAME_DATA_DIR}/board.pdf")
# board_ref = cv.imread(f"{GAME_DATA_DIR}/board.jpg")

simshow(board_ref)

## Milestone 1

In this phase, the following things were detected:
- the black dice tray along with the dice on it
- the board

### Dice tray detection

In [ ]:
image = FIRST_FRAMES["easy"]
# image = FIRST_FRAMES["medium"]
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [ ]:
mini_imshow(gray)

The dice tray is all black so a simple threshold was performed.

In [ ]:
tray_cont,dice1_cont,dice2_cont,img_cont = detect_dice_tray(image, 30)
imshow(img_cont)

In [ ]:
tray = crop_contour(image, tray_cont)

dice1 = crop_contour(image, dice1_cont)
dice2 = crop_contour(image, dice2_cont)

mini_imshow(tray)
imshow(dice1)
imshow(dice2)

### Board detection

Detecting the board was harder as it has much more details.

In [ ]:
board_gray = cv.GaussianBlur(cv.cvtColor(board_ref, cv.COLOR_BGR2GRAY),(7,7),0)

simshow(board_gray)

To achieve this steps descriptors are used, in particular the SIFT detector. To quickly match the descriptors FLANN algorithm is used.

In [ ]:
M_board, drawn_matches_board, board_cont = descriptor_detect(image, board_ref)
simshow(drawn_matches_board)

In [ ]:
crop_board = crop_contour(image, board_cont)
crop_board = rotate_image(crop_board,0.8)
simshow(crop_board)

This code doesn't have to be run much, because the board should not a lot move in the clips

In the milestone 1, there were also attempts to segment the image using a Gaussian Mixture, but they were quite slow and not effective

## Further Progress

### Tracking Game Score

Game score is tracked in the lower half of the board, by blue and orange counters.

In [ ]:
simshow(np.concatenate([board_ref, board_mask], axis=1))

In [ ]:
cell_contours, score_cont = detect_score_board(board_ref, board_mask[:,:,2])
score_x,score_y,_,_ = cv.boundingRect(score_cont)
test_score_crop = crop_contour(board_ref,score_cont)
imshow(test_score_crop)

In [ ]:
cell_contours = list(map(lambda cont: warp_contour(cont, M_board), [cont + [score_x,score_y] for cont in cell_contours]))
simshow(cv.drawContours(np.copy(image), cell_contours, -1,(255,0,0),2))

In [ ]:
imshow(crop_contour(image, cell_contours[0]))

In [ ]:
for i, cell in enumerate(cell_contours):
    print(f"Cell {i}: ", end=" ")
    print(f"Orange: {calculate_color_percentage(crop_contour(image, cell),LOWER_ORANGE,UPPER_ORANGE):.2f}%", end=" ")
    print(f"Blue: {calculate_color_percentage(crop_contour(image, cell),LOWER_DARK_BLUE,UPPER_DARK_BLUE):.2f}%")

In [ ]:
calculate_current_score(image, cell_contours, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))

### Tracking Card Pile

In [ ]:
card_ref = read_pdf(f"{GAME_DATA_DIR}/card_reverse.pdf")
imshow(card_ref)

In [ ]:
M_card, drawn_matches_card, card_cont = descriptor_detect(image, card_ref, distance=0.5)

In [ ]:
mini_imshow(drawn_matches_card)

In [ ]:
crop_card = crop_contour(image, card_cont)
imshow(crop_card)

### Tracking Buildings

In [ ]:
building_contours = detect_buildings(board_mask[:,:,0])
building_contours = list(map(lambda cont: warp_contour(cont, M_board), building_contours))
simshow(cv.drawContours(np.copy(image), building_contours, -1,(255,0,0),2))

In [ ]:
for i, building in enumerate(building_contours):
    b_crop = crop_contour(image, building)
    print(f"Building {i}: ", end=" ")
    print(f"Orange: {calculate_color_percentage(b_crop,LOWER_ORANGE,UPPER_ORANGE):.2f}%", end=" ")
    print(f"Blue: {calculate_color_percentage(b_crop,LOWER_DARK_BLUE,UPPER_DARK_BLUE):.2f}%")
    imshow(b_crop)

## Putting it all together

In [ ]:
IGNORE_DIR = "./ignore"

In [ ]:
easy = cv.VideoCapture(f"{CLIP_DIRS['easy']}/{clip_mp4(0)}")
if easy.isOpened():
    print("Video loaded")

width,height = int(easy.get(3)), int(easy.get(4))

print(width, height)

fps = easy.get(cv.CAP_PROP_FPS)
print(fps)

In [ ]:
start = 0
easy.set(cv.CAP_PROP_POS_FRAMES,start)
ret, frame = easy.read()

In [ ]:
board = Board("board", board_ref)
card = Card("cards", "CSRT", card_ref)
card_pile = CardPile("card_pile", card_ref)
score_board = ScoreBoard("score_board", board, board_mask[:, :, 2])
buildings = Buildings("buildings", board_mask[:, :, 0]) 
dice_1 = Dice("dice_1", "CSRT", 1, threshold=30)
dice_2 = Dice("dice_2", "CSRT", 2, threshold=30)

In [ ]:
track_video(easy, f"{IGNORE_DIR}/{clip(0)}", board, [card, dice_1, dice_2], [card_pile, score_board, buildings], sec=60)

In [ ]:
for diff in DIFFICULTIES:
    for i in range(3):
        clip_name = f"{diff}_{clip(i)}"
        print(f"Processing {clip_name}")
        video = cv.VideoCapture(f"{RESIZED_CLIP_DIRS[diff]}/{clip_mp4(i)}")
        
        if video.isOpened():
            print("Video loaded")
            
        _, frame = video.read()


        card = Card("cards", "CSRT", card_cont_resized, frame)
        card_pile = CardPile("card_pile", card_cont_resized,card_ref)
        score_board = ScoreBoard("score_board", list(map(resize_contour, cell_contours)),board_ref, board_mask[:, :, 2], frame, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))
        buildings = Buildings("buildings",  list(map(resize_contour, building_contours)), board_mask[:, :, 0], frame, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))
        board = Board("board",board_cont,board_ref)

        
        track_video(video, f"{IGNORE_DIR}/{clip_name}",board, [card], [card_pile, score_board, buildings], sec=60)